In [1]:
%load_ext autoreload
%autoreload 2

## Brenden APS Mesoporous silica nanoparticles data loading

Requires Brenden's saxs_data_processing package to load Xenocs data files. Install here: https://github.com/pozzo-research-group/saxs_data_processing. Needs the `io` module.


In [2]:
from saxs_data_processing import io
import os
import pandas as pd
import numpy as np
import json
import glob


## Load constants and filepaths

There are 2 sets of SAXS data: Low-Q data collected at APS, and high-Q data measured at UW 11 days later. The UW data was collected on the same samples made at APS. This was done b/c the SAXS camera at APS was down. These sets of data are referred to as 'saxs' and 'usaxs' herein.

Link to data: (must be added to share): https://drive.google.com/drive/folders/1E3q6wa394yXFD1YeCJbNZqaGLOP3RNGE?usp=drive_link

In [3]:
# Update this to point at the "Phasemap_data" directory you downloaded from google drive
data_root_fp = "/mnt/c/Users/bgpel/My Drive/Research/PozzoResearchGroup/APS_Beamtime_Sp25/Phasemap_data"

In [4]:
saxs_fp = f'{data_root_fp}/saxs' 
usaxs_fp = f'{data_root_fp}/usaxs' 

In [5]:
saxs_paths = os.listdir(saxs_fp)
usaxs_paths = os.listdir(usaxs_fp)

In [6]:
sample_table = pd.read_csv(f'{data_root_fp}/APS_sample_reference_cartridgebatch.csv')

In [7]:
composition_table = pd.read_csv(f'{data_root_fp}/Mesoporous_SobolBaseline_APS_BatchMode_3_29_25.csv').set_index('uuid', drop=True)

In [8]:
with open(f'{data_root_fp}/Mesoporous_constants_APS_batch.json', 'rt') as f:
    constants = json.load(f)

In [9]:
def get_composition(row, teos_dilution_ratio, ctab_concentration, f127_concentration):
    teos_volume = row['teos_volume']
    ammonia_volume = row['ammonia_volume']
    water_volume = row['water_volume']
    ethanol_volume = row['ethanol_volume']
    ctab_volume = row['ctab_volume']
    F127_volume = row['F127_volume']

    total_volume = teos_volume + ammonia_volume + water_volume + ethanol_volume + ctab_volume + F127_volume
    print('Total volume: ', total_volume)
    
    ammonia_vf = ammonia_volume/total_volume
    water_delivered_vf = water_volume / total_volume
    ethanol_delivered_vf = ethanol_volume/total_volume
    ctab_vf = ctab_volume/total_volume
    F127_vf = F127_volume/total_volume


    ## Need to correct for mixed compositions

    # TEOS and ethanol:
    teos_vf = teos_volume/teos_dilution_ratio/total_volume
    teos_etoh_vf = teos_volume*((teos_dilution_ratio - 1)/teos_dilution_ratio)/total_volume

    ethanol_vf = ethanol_delivered_vf + teos_etoh_vf

    print('Ethanol delivered vf: ', ethanol_delivered_vf)
    print('Ethanol TEOS vf: ', teos_etoh_vf)
    print('Total ethanol vf: ', ethanol_vf)

    

    water_vf = water_delivered_vf + F127_vf + ctab_vf

    ctab_mass = ctab_volume * ctab_concentration
    f127_mass = F127_volume * f127_concentration

    # 

    return {'teos_vf':teos_vf, 'ammonia_vf':ammonia_vf, 'water_vf':water_vf, 'ethanol_vf':ethanol_vf, 'ctab_mass':ctab_mass, 'F127_mass':f127_mass}

In [10]:
def get_filename(stub, paths):
    for path in paths:
        if stub in path:
            return path

In [ ]:
mesoporous_samples = {}
teos_dilution_ratio = constants['TEOS']['dilution_ratio']
ctab_concentration = constants['ctab']['stock_concentration_mg_uL']
f127_concentration = constants['f127']['stock_concentration_mg_uL']

for i, row in sample_table.iloc[0:73].iterrows(): #0:73 selected to not load samples that were not synthesized
    sample = {}
    sample['sample_name'] = row['Sample name']
    sample['cartridge_number'] = row['Cartridge plate']
    sample['cartridge_cell'] = row['Cartridge cell']
    sample_uid = row['Sample ID']


    if sample_uid is not np.nan:
        
        saxs_filename = glob.glob(os.path.join(saxs_fp, f'*{sample_uid}*'))
        usaxs_filename = glob.glob(os.path.join(usaxs_fp, f'*{sample_uid}*'))
        print(saxs_filename)
        if len(saxs_filename) == 0:
            print(f'Warning: No files found for sample {sample_uid}')
            continue
            
        sample['saxs_data'] = io.read_1D_data(saxs_filename[0])
        sample['usaxs_data'] = pd.read_csv(usaxs_filename[0], sep = '\t')


        composition_row = composition_table.loc[sample_uid]
        composition = get_composition(composition_row, teos_dilution_ratio, ctab_concentration, f127_concentration)
        sample['composition'] = composition
        mesoporous_samples[sample_uid] = sample


        


        
    

[]
['/mnt/c/Users/bgpel/My Drive/Research/PozzoResearchGroup/APS_Beamtime_Sp25/Phasemap_data/saxs/04247_A4-T2_93780a39-35f5-4c85-963b-2fde66396c78_000_sub_merged_merged.dat']
Total volume:  9999.999992000001
Ethanol delivered vf:  0.2721331681177065
Ethanol TEOS vf:  0.03215435182396909
Total ethanol vf:  0.3042875199416756
['/mnt/c/Users/bgpel/My Drive/Research/PozzoResearchGroup/APS_Beamtime_Sp25/Phasemap_data/saxs/04250_A5-T2_00afc63f-86d3-41a0-9f37-8d56c6d1abfd_000_sub_merged_merged.dat']
Total volume:  9999.999992000001
Ethanol delivered vf:  0.2721331681177065
Ethanol TEOS vf:  0.03215435182396909
Total ethanol vf:  0.3042875199416756
['/mnt/c/Users/bgpel/My Drive/Research/PozzoResearchGroup/APS_Beamtime_Sp25/Phasemap_data/saxs/04251_A6-T2_83caa81b-c9ac-4b2f-b7b6-97a05c42f277_000_sub_merged_merged.dat']
Total volume:  10000.0
Ethanol delivered vf:  0.2101010853994405
Ethanol TEOS vf:  0.053794442619034086
Total ethanol vf:  0.2638955280184746
['/mnt/c/Users/bgpel/My Drive/Researc

mesoporous_samples is a dictionary containing composition and scattering information for each sample. It's structure is:

sample_uid:
- sample_name
- cartridge_number
- cartridge_cell
- saxs_data (tuple, (Pandas dataframe of scattering, metadata))
- usaxs_data (Pandas dataframe with Q, I, ISigma)
- composition - dictionary of component name to value